# Cards Power Analysis
This is a bit outside the scope of what we will ask you to do in this class, but it's a very real example.

In class, Jason had a deck of playing cards that did not have the typical proportions of each suit. As such, he knew from the beginning that the null hypothesis (this deck has an equal proportion of each suit) is actually false (something we rarely know in real-world problems). But, we failed to reject the null! So we made a type II error.

With a normal deck of cards, if we shuffled and took samples of 30 cards, computed the chi-squared value like we did in class, and compared it to the critical value, we'd make a false positive about 5% of the time (our significance level is 0.05, so we have a 5% chance of a type I error). **

With Jason's deck, we know the true proportions of each suit, so we know what samples from that deck will look like, and we can compute (or here, I actually simulate rather than doing the math formally) the probability that we make a type II error.


** Technically, the value is probably a little different because our sample of cards is done without replacement, but all the math assumes that it's done with replacement, so those values aren't quite exact, but they're certainly going to get us in the right direction.

In [50]:
# Stuff to make the code work
import numpy as np
import scipy.stats as ss

In [53]:
# True population (for generating samples)
pop = np.array([0.375, 0.375, 0.125, 0.125])

# Number of samples to take
N_samples = 100000
# Number of cards in each sample
N_cards = 30
# Generate samples
samples = np.random.choice(range(4), p=pop, replace=True, size=(N_samples, N_cards))

# Chi-squared critical value for df=4-1, alpha=0.05
CV = ss.chi2.ppf(0.95, 3)

# Store our results
results = np.zeros(N_samples)

# Under "null" hypothesis (all equal fraction), what are the expected counts?
fair_expected = np.array([0.25, 0.25, 0.25, 0.25]) * N_cards

# For each sample, compute the chi-squared value
for i in range(N_samples):
    sample = samples[i, :]
    suits = np.zeros(4)
    for j in range(4):
        suits[j] = np.sum(sample == j)
    chi_square = np.sum((suits - fair_expected) ** 2 / fair_expected)
    results[i] = chi_square
    
# Percentage of samples that are below the critical value
# yielding a type II error (false negative)
print("Probability of type II error: %.2f%%" % (np.sum(results < CV) / float(N_samples) * 100))

Probability of type II error: 34.80%


So there's about a 35% chance of getting a type II error in Jason's deck. Meaning, 35% of the time, we'll get a sample that looks close enough to a sample from a normal deck of cards that we simply can't tell the difference, and fail to reject the null.

If we had a slightly different population of cards 40-40-10-10, we would have a different type II error:

In [54]:
# True population (for generating samples)
pop = np.array([0.4, 0.4, 0.1, 0.1])

# Number of samples to take
N_samples = 100000
# Number of cards in each sample
N_cards = 30
# Generate samples
samples = np.random.choice(range(4), p=pop, replace=True, size=(N_samples, N_cards))

# Chi-squared critical value for df=4-1, alpha=0.05
CV = ss.chi2.ppf(0.95, 3)

# Store our results
results = np.zeros(N_samples)

# Under "null" hypothesis (all equal fraction), what are the expected counts?
fair_expected = np.array([0.25, 0.25, 0.25, 0.25]) * N_cards

# For each sample, compute the chi-squared value
for i in range(N_samples):
    sample = samples[i, :]
    suits = np.zeros(4)
    for j in range(4):
        suits[j] = np.sum(sample == j)
    chi_square = np.sum((suits - fair_expected) ** 2 / fair_expected)
    results[i] = chi_square
    
# Percentage of samples that are below the critical value
# yielding a type II error (false negative)
print("Probability of type II error: %.2f%%" % (np.sum(results < CV) / float(N_samples) * 100))

Probability of type II error: 15.57%


So when the population is even more different from the null hypothesis than it was in class, we'd only make a type II error about 16% of the time. That's less than half of what it was before!

What if the population was a bit closer to the null hypothesis: 33.3-33.3-16.7-16.7?

In [55]:
# True population (for generating samples)
pop = np.array([1.0/3.0, 1.0/3.0, 1.0/6.0, 1.0/6.0])

# Number of samples to take
N_samples = 100000
# Number of cards in each sample
N_cards = 30
# Generate samples
samples = np.random.choice(range(4), p=pop, replace=True, size=(N_samples, N_cards))

# Chi-squared critical value for df=4-1, alpha=0.05
CV = ss.chi2.ppf(0.95, 3)

# Store our results
results = np.zeros(N_samples)

# Under "null" hypothesis (all equal fraction), what are the expected counts?
fair_expected = np.array([0.25, 0.25, 0.25, 0.25]) * N_cards

# For each sample, compute the chi-squared value
for i in range(N_samples):
    sample = samples[i, :]
    suits = np.zeros(4)
    for j in range(4):
        suits[j] = np.sum(sample == j)
    chi_square = np.sum((suits - fair_expected) ** 2 / fair_expected)
    results[i] = chi_square
    
# Percentage of samples that are below the critical value
# yielding a type II error (false negative)
print("Probability of type II error: %.2f%%" % (np.sum(results < CV) / float(N_samples) * 100))

Probability of type II error: 69.51%


We're now going to make a type II error almost 70% of the time!!!

What we can take away from this is:
* It's not so surprising that we got a type II error in class (it's about the same as rolling one dice and getting a 1 or 2, which we can hardly call "unusual")
* The probability of getting a type II error goes DOWN when the true population is very far from the null hypothesis
* The probability of getting a type II error goes UP when the true population is very close to the null hypothesis (but not exactly the null)

And, remembering what we learned previously, the power, which is one minus the probability of a type II error, will go in the opposite direction:
* The power goes UP when the true population is far from the null
* The power goes DOWN when the true population is close to the null

So this is why we don't have you compute the probability of a type II error in class for real-world problems. The probability of a type I error is a really well-defined quantity: the probability of a false positive when the null is in fact true. But the probability of a type II error depends on the effect size -- just how different is the true population from what the null hypothesis assumes? In some settings, we might be able to do everything just with math, but sometimes we need to run simulation studies (like above) to look at different plausible scenarios, determine our power, and see if we think we stand a good chance of rejecting a false null.

For one last example, an easy way to increase the power (decrease the chance of a type II error) is to increase the sample size. Let's try a sample of 60 cards instead with Jason's deck:

In [59]:
# True population (for generating samples)
pop = np.array([0.375, 0.375, 0.125, 0.125])

# Number of samples to take
N_samples = 100000
# Number of cards in each sample
N_cards = 60
# Generate samples
samples = np.random.choice(range(4), p=pop, replace=True, size=(N_samples, N_cards))

# Chi-squared critical value for df=4-1, alpha=0.05
CV = ss.chi2.ppf(0.95, 3)

# Store our results
results = np.zeros(N_samples)

# Under "null" hypothesis (all equal fraction), what are the expected counts?
fair_expected = np.array([0.25, 0.25, 0.25, 0.25]) * N_cards

# For each sample, compute the chi-squared value
for i in range(N_samples):
    sample = samples[i, :]
    suits = np.zeros(4)
    for j in range(4):
        suits[j] = np.sum(sample == j)
    chi_square = np.sum((suits - fair_expected) ** 2 / fair_expected)
    results[i] = chi_square
    
# Percentage of samples that are below the critical value
# yielding a type II error (false negative)
print("Probability of type II error: %.2f%%" % (np.sum(results < CV) / float(N_samples) * 100))

Probability of type II error: 5.82%


Ta-da! Just by taking a bigger sample (with the same population as in class), we have reduced the probability of a type II error from 35% to about 6% (and thus increase the power from 65% to 94%)!